# Modelo de Redes Neuronales

En el presente notebook se construye un modelo de Red Neuronal a partir del conjunto de datos que sigue la estructura de la tabla mostrada abajo. Para esta tabla **blue**, **green**, **red**, **nir**, **swir1** y **swir2** son las características de los pixeles de una escena landsat y **wofs**, **bosque**, **wofs_bosque**,  **ninguno**, **cloud** son las clases a la cual pertence casa pixel. Un pixel solo pertenece a una única clase.
El modelo generado es guardado en el archivo **model.h5** para su posterior uso en otros notebooks.


| blue | green | red | nir | swir1 | swir2 | wofs | bosque | wofs_bosque | ninguno  | cloud |
|------|-------|-----|-----|-------|-------|------|--------|-------------|----------|-------|
|      |       |     |     |       |       |      |        |             |          |       |


In [13]:
%matplotlib inline

from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras import regularizers
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import classification_report

import keras
import matplotlib.pyplot as plt
from rasterio.plot import show
import xarray as xr

import rasterio
import glob
import os
import numpy as np
import pandas as pd

## Lectura del Dataset por URL

In [2]:
data = pd.read_csv('https://docs.google.com/uc?export=download&id=1zIf10M5nkLdneCYU1Kw_KyEOynNp3hb2')
data.head()

,blue,green,red,nir,swir1,swir2,wofs,bosque,wofs_bosque,ninguno,cloud
0,8452.0,8750.0,8900.0,9146.0,6759.0,4585.0,0,0,0,0,1
1,299.0,203.0,100.0,73.0,79.0,60.0,1,0,0,0,0
2,5167.0,4956.0,4945.0,5647.0,3920.0,2588.0,0,0,0,0,1
3,245.0,177.0,93.0,75.0,100.0,80.0,1,0,0,0,0
4,259.0,179.0,92.0,64.0,89.0,72.0,1,0,0,0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 11 columns):
blue           1000000 non-null float64
green          1000000 non-null float64
red            1000000 non-null float64
nir            1000000 non-null float64
swir1          1000000 non-null float64
swir2          1000000 non-null float64
wofs           1000000 non-null int64
bosque         1000000 non-null int64
wofs_bosque    1000000 non-null int64
ninguno        1000000 non-null int64
cloud          1000000 non-null int64
dtypes: float64(6), int64(5)
memory usage: 83.9 MB


Se preparan los datos separando las characteristicas de los pixeles y las etiquetas de los pixeles. Luego se separan los datos en datos de entrenamiento y datos de prueba.

In [4]:
dataset_X = data.drop(columns=['wofs','bosque','wofs_bosque','ninguno','cloud'], axis=1)
dataset_Y = data[['wofs','bosque','wofs_bosque','ninguno','cloud']]

X_train, X_test, y_train, y_test = train_test_split(dataset_X, dataset_Y, test_size=0.2)
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

## Definición del Modelo

Se experimento con varios modelos distintos. Inicialmente se probó con modelos de una sola capa con una neurona, 4 neuronas, 6 neuronas, y 9 neuronas, agregando más neuronas después de este punto no se obtuvieron mejoras significativas. Luego se experimentó agregando más capas al modelo, la exactitud mejoró significativamente hasta la tercera capa. 

Se utilizó la función de activacion ReLU en todas las capas. La función softmax en la capa de salida asegura una clasificación mutuamente excluyente.

In [5]:
model = keras.Sequential([
    keras.layers.Dense(9,input_dim=6, activation='relu'),
    keras.layers.Dense(9, activation='relu'),
    keras.layers.Dense(9, activation='relu'),
    keras.layers.Dense(5, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
# Resumen de parámetros del modelo
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 9)                 63        
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 90        
_________________________________________________________________
dense_3 (Dense)              (None, 9)                 90        
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 50        
Total params: 293
Trainable params: 293
Non-trainable params: 0
_________________________________________________________________


In [6]:
# ENTRENAMIENTO
model.fit(X_train, y_train, epochs=2)

Epoch 1/2
800000/800000 [==============================] - 11s 14us/step - loss: 0.4425 - accuracy: 0.9566
Epoch 2/2
800000/800000 [==============================] - 11s 14us/step - loss: 0.0836 - accuracy: 0.9691


In [7]:
print("Saving model...")
model.save("model.h5")

Saving model...


## Resultados

* Clase 0 (wofs): Agua
* Clase 1 (bosque): Bosque
* Clase 2 (wofs_bosque): Agua o Bosque
* Clase 3 (cloud): Nubes
* Clase 4 (ninguno): Ninguna de las coverturas anteriores. Puedes ser repreentar una urbanización.

In [8]:
# EVALUACIÓN
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)


Test accuracy: 0.9689499735832214


In [9]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred,axis=1)
y_pred_classes

array([1, 1, 1, ..., 0, 0, 1])

In [10]:
y_test_classes = np.argmax(y_test,axis=1)
y_test_classes

array([1, 1, 1, ..., 0, 0, 1])

In [11]:
confusion_matrix(y_test_classes, y_pred_classes)

array([[67435,     0,     0,   230,   269],
       [    9, 41094,     0,  2101,    90],
       [   31,     0,     0,    10,     2],
       [   81,   656,     0,  6293,  2082],
       [   14,     9,     0,   626, 78968]])

In [14]:
print(classification_report(y_test_classes, y_pred_classes))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     67934
           1       0.98      0.95      0.97     43294
           2       0.00      0.00      0.00        43
           3       0.68      0.69      0.69      9112
           4       0.97      0.99      0.98     79617

    accuracy                           0.97    200000
   macro avg       0.73      0.72      0.73    200000
weighted avg       0.97      0.97      0.97    200000



## Análisis y Conclusiones

El modelo entrenado logra una exactitud del 96% sobre los datos de prueba. Comparandolo con el modelo sin la clase "cloud", este modelo logra una menor exactitud causado por una menor representación en los datos para las otras clases. La clase que fué más afectada fue "ninguno".

Las clases de "wofs", "bosque", y "cloud" son las mejores representadas en el dataset, y consequentement, logra mejores resultados en las predicciones del modelo, con una precisión del 98%.

La clase "ninguno" logra una precisión del 69%, y un recall del 75%, valores significativamente menores que aquellos obtenidos en el modelo sin nubes. A pesar de su poca representación en los datos, esta clase obtuvo metricas relativamente altas, estó parece indicar que esta clase posee caracteristicas faciles de identificar(Con valores del espectro de luz significativamente diferentes a los del bosque o el agua). Sin embargo, considerando que esta clase representa muchos terrenos, es posible que el modelo no generalice bien esta clase con otras imagenes satelitales.

Al igual que el modelo sin nubes, la clase wofs_bosque, siendo la menos representada, no logra ser predecida por el modelo. Esto se evidencia en una precisión y un recall igual a 0.

Para solucionar la poca representación de wofs_bosque, se podría buscar una imagen satelital con mayor superposición entre bosque y agua, como, por ejemplo, manglares, pantanos, o lagunas.